In [1]:
!pip install konlpy
!pip install wordcloud

#import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from bokeh.plotting import figure
import random
from konlpy.tag import Komoran
tagger = Komoran()
from collections import Counter
from wordcloud import WordCloud
import networkx as nx
import re
import math

In [2]:
with open('movie_df.xlsx',mode="rb") as excel_file:
    DF = pd.read_excel(excel_file)

In [3]:
DF

title  year  audience                 date        genre  \
0                  범죄도시 2  2022  12693023  2022-05-18 00:00:00       범죄, 액션   
1                 탑건: 매버릭  2022   8171923  2022-06-22 00:00:00           액션   
2               한산: 용의 출현  2022   7262885  2022-07-27 00:00:00      액션, 드라마   
3              공조2: 인터내셔날  2022   6948819  2022-09-07 00:00:00      액션, 코미디   
4     닥터 스트레인지: 대혼돈의 멀티버스  2022   5884600  2022-05-04 00:00:00  액션, 판타지, 모험   
...                   ...   ...       ...                  ...          ...   
4291               무림 악인전  1980     20342  1980-04-10 00:00:00           액션   
4292                   복권  1980     20307  1980-06-21 00:00:00          NaN   
4293             바다로 간 목마  1980     20041  1980-10-18 00:00:00          NaN   
4294           최인호의 병태 만세  1980     18332  1980-12-12 00:00:00          NaN   
4295                해뜨는 집  1980     17987  1980-12-06 00:00:00          NaN   

                                                  actor country  \
0           마동석(마석도), 손석구(강해상), 최귀화(전일만), 마동석, 손석구, 최귀화    대한민국   
1     톰 크루즈(매버릭), 마일즈 텔러(루스터), 제니퍼 코넬리(페니), 톰 크루즈, 마...      미국   
2     박해일(이순신), 변요한(와키자카 야스하루), 안성기(어영담), 박해일, 변요한, 안성기    대한민국   
3               현빈(림철령), 유해진(강진태), 윤아(박민영), 현빈, 유해진, 윤아    대한민국   
4     베네딕트 컴버배치(닥터 스트레인지), 엘리자베스 올슨(완다 막시모프 / 스칼렛 위치...      미국   
...                                                 ...     ...   
4291                       정진화, 서영란, 장일식, 정진화, 서영란, 장일식    대한민국   
4292                       정진화, 김명아, 김기주, 정진화, 김명아, 김기주    대한민국   
4293                                 장미희, 윤일봉, 장미희, 윤일봉    대한민국   
4294                       엄인태, 이미영, 정한용, 엄인태, 이미영, 정한용    대한민국   
4295                       강주희, 이승현, 김정훈, 강주희, 이승현, 김정훈    대한민국   

              director      age  score  \
0             이상용, 이상용  15세 관람가   9.32   
1     조셉 코신스키, 조셉 코신스키  12세 관람가   9.76   
2             김한민, 김한민  12세 관람가   8.75   
3             이석훈, 이석훈  15세 관람가   8.44   
4         샘 레이미, 샘 레이미  12세 관람가   7.22   
...                ...      ...    ...   
4291      김정용, 김정용, 신고      NaN   7.00   
4292          김정용, 김정용  12세 관람가   0.00   
4293      정진우, 정진우, 신고  12세 관람가   6.29   
4294          김수형, 김수형  12세 관람가   0.00   
4295      석래명, 석래명, 신고   전체 관람가   9.44   

                                               synopsis  
0     ['가리봉동 소탕작전 후 4년 뒤,금천서 강력반은 베트남으로 도주한 용의자를 인도받...  
1     ['최고의 파일럿이자 전설적인 인물 매버릭(톰 크루즈)은 자신이 졸업한 훈련학교 교...  
2     ['1592년 4월, 조선은 임진왜란 발발 후 단 15일 만에 왜군에 한양을 빼앗기...  
3     ['공조 이즈 백! 이번엔 삼각 공조다!남한으로 숨어든 글로벌 범죄 조직을 잡기 위...  
4     ['끝없이 균열되는 차원과 뒤엉킨 시공간의 멀티버스가 열리며오랜 동료들, 그리고 차...  
...                                                 ...  
4291  ['고구려가 요동땅을 다스릴때 막대한 황금엽전을 그곳에 매장한다. 고려왕조가 기울어...  
4292  ['이대복은 신체조건이 맞지않아 공초대사로부터 다른길을 가라는 가르침을 받고 하산한...  
4293  ['여대생 주희(장미희 분)는 11년 연상의 농아학교 교사 이민우(윤일봉 분)를 사...  
4294  ['신학기가 시작되자 철학과 강의실에 편입생 순철이 들어오므로 병태와 친숙해진다. ...  
4295  ['영아는 말괄량이로 엄마와 둘이 살아간다. 영아는 친구들과 어울려 미팅장소엘 갔다...  

[4296 rows x 11 columns]

In [4]:
DF.columns = ['title','year','audience','date','genre','actor','country','director','age','score','synopsis']
DF["title"]=DF["title"].apply(str)
DF["genre"]=DF["genre"].apply(str)
DF["actor"]=DF["actor"].apply(str)
DF["director"]=DF["director"].apply(str)
DF["age"] = DF["age"].apply(str)

In [5]:
DF['genre']=DF['genre'].replace(["멜로"],'로맨스')

In [6]:
DF[DF['year']==2022]

title  year  audience                 date         genre  \
0                범죄도시 2  2022  12693023  2022-05-18 00:00:00        범죄, 액션   
1               탑건: 매버릭  2022   8171923  2022-06-22 00:00:00            액션   
2             한산: 용의 출현  2022   7262885  2022-07-27 00:00:00       액션, 드라마   
3            공조2: 인터내셔날  2022   6948819  2022-09-07 00:00:00       액션, 코미디   
4   닥터 스트레인지: 대혼돈의 멀티버스  2022   5884600  2022-05-04 00:00:00   액션, 판타지, 모험   
..                  ...   ...       ...                  ...           ...   
95              어나더 라운드  2022     36126  2022-01-19 00:00:00           드라마   
96            배니싱: 미제사건  2022     35164  2022-03-30 00:00:00       범죄, 스릴러   
97                뒤틀린 집  2022     34886  2022-07-13 00:00:00  공포, 드라마, 스릴러   
98                   봄날  2022     34100  2022-04-27 00:00:00  드라마, 액션, 느와르   
99          아치의 노래, 정태춘  2022     33399  2022-05-18 00:00:00         다큐멘터리   

                                                actor country  \
0         마동석(마석도), 손석구(강해상), 최귀화(전일만), 마동석, 손석구, 최귀화    대한민국   
1   톰 크루즈(매버릭), 마일즈 텔러(루스터), 제니퍼 코넬리(페니), 톰 크루즈, 마...      미국   
2   박해일(이순신), 변요한(와키자카 야스하루), 안성기(어영담), 박해일, 변요한, 안성기    대한민국   
3             현빈(림철령), 유해진(강진태), 윤아(박민영), 현빈, 유해진, 윤아    대한민국   
4   베네딕트 컴버배치(닥터 스트레인지), 엘리자베스 올슨(완다 막시모프 / 스칼렛 위치...      미국   
..                                                ...     ...   
95  매즈 미켈슨, 토머스 보 라센, 라르스 란데, 매즈 미켈슨, 토머스 보 라센, 라르...     덴마크   
96  유연석(박진호), 올가 쿠릴렌코(알리스 로네), 예지원(이미숙), 유연석, 올가 쿠...     프랑스   
97           서영희(명해), 김보민(희우), 김민재(현민), 서영희, 김보민, 김민재    대한민국   
98                                       손현주(호성), 손현주    대한민국   
99                         정태춘(본인), 박은옥(본인), 정태춘, 박은옥    대한민국   

                    director      age  score  \
0                   이상용, 이상용  15세 관람가   9.32   
1           조셉 코신스키, 조셉 코신스키  12세 관람가   9.76   
2                   김한민, 김한민  12세 관람가   8.75   
3                   이석훈, 이석훈  15세 관람가   8.44   
4               샘 레이미, 샘 레이미  12세 관람가   7.22   
..                       ...      ...    ...   
95  토마스 빈터베르그, 토마스 빈터베르그, 신고  15세 관람가   8.38   
96          드니 데르쿠르, 드니 데르쿠르  15세 관람가   4.05   
97                  강동헌, 강동헌  15세 관람가   4.17   
98                  이돈구, 이돈구  15세 관람가   7.56   
99                  고영재, 고영재   전체 관람가   9.35   

                                             synopsis  
0   ['가리봉동 소탕작전 후 4년 뒤,금천서 강력반은 베트남으로 도주한 용의자를 인도받...  
1   ['최고의 파일럿이자 전설적인 인물 매버릭(톰 크루즈)은 자신이 졸업한 훈련학교 교...  
2   ['1592년 4월, 조선은 임진왜란 발발 후 단 15일 만에 왜군에 한양을 빼앗기...  
3   ['공조 이즈 백! 이번엔 삼각 공조다!남한으로 숨어든 글로벌 범죄 조직을 잡기 위...  
4   ['끝없이 균열되는 차원과 뒤엉킨 시공간의 멀티버스가 열리며오랜 동료들, 그리고 차...  
..                                                ...  
95  ['각각 역사, 체육, 음악, 심리학을 가르치는 같은 고등학교 교사 니콜라이, 마르...  
96  ['어느 날 심하게 훼손되어 신원을 알 수 없는 변사체가 발견되고사건을 맡은 형사 ...  
97  ['피치 못할 사정으로 외딴집에 이사 오게 된 가족.엄마 ‘명혜’는 이사 온 첫 날...  
98  ["한때는 잘나가던 큰형님 '호성'(손현주).8년 만에 출소해 보니 남보다 못한 동...  
99  ['10대 가수상, 가요 사전심의 철폐운동그리고 음악시장을 홀연히 떠나기까지어디서도...  

[96 rows x 11 columns]

In [7]:
# 장르 태깅(명사, 고유명사)
# 멜로와 로맨스는 항상 같이 나와서 로맨스로 통일하고 멜로를 없앰
genre_list = {}
genre_list_all = []
for i in range(0,43):
    genre_list[i] = []
    sentences = DF[DF.year==2022-i]['genre'].values.tolist()
    tagged_sentences = [tagger.pos(sent) for sent in sentences]
    for sent in tagged_sentences:  
        for word, tag in sent:
            if tag in ['NNG','NNP'] and len(word)>1 and word not in ["멜로"]:
                genre_list[i].append(word)
                genre_list_all.append(word)

In [8]:
top_genre = []
for i in range(len(Counter(genre_list_all))):
    top_genre.append(Counter(genre_list_all).most_common()[i][0])

In [9]:
top_genre

['드라마',
 '액션',
 '코미디',
 '스릴러',
 '로맨스',
 '모험',
 '범죄',
 '판타지',
 '애니메이션',
 '가족',
 '공포',
 '미스터리',
 '전쟁',
 '뮤지컬',
 '다큐멘터리',
 '느와르',
 '서부',
 '에로',
 '서스펜스',
 '서사',
 '실험',
 '무협']

# 전쟁영화

In [ ]:
#전쟁 영화 

DF_war=DF[DF.genre.str.contains('전쟁')]
DF_war

In [ ]:
# 전쟁영화 시놉시스 태깅(명사)
war_synopsis_list = {}
for i in range(2022,1979,-1):
    war_synopsis_list[i] = []
    sentences = DF_war[DF.year==i]['synopsis'].values.tolist()
    tagged_sentences = [tagger.pos(sent) for sent in sentences]
    for sent in tagged_sentences:   
        for word, tag in sent:
            if tag in ['NNG'] and word not in ['시작','한편','자신']: #and len(word)>1
                war_synopsis_list[i].append(word)


In [ ]:
# 리스트 틀 만들기
war_synopsis_info = {
    "연도": [],
    "1위 키워드": [],
    "2위 키워드": [],
    "3위 키워드": [],
    "4위 키워드": [],
    "5위 키워드": [],
    "6위 키워드": [],
    "7위 키워드": [],
    "8위 키워드": [],
    "9위 키워드": [],
    "10위 키워드": []
}
war_si = pd.DataFrame(war_synopsis_info)

In [ ]:
war_si

In [ ]:
for i in range(2022,1979,-1):
    if sum(Counter(war_synopsis_list[i]).values())==0:
        print(i)

In [ ]:
for i in range(2022,1979,-1):
    if i in [2021,2013,2006]:
        i=i-1
        
    synopsis_data = Counter(war_synopsis_list[i]).most_common()
    synopsis_m = sum(Counter(war_synopsis_list[i]).values())  
    new_data = {
        "연도": str(i),
        "1위 키워드": synopsis_data[0][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[0][1]/synopsis_m*100) + ")",
        "2위 키워드": synopsis_data[1][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[1][1]/synopsis_m*100) + ")",
        "3위 키워드": synopsis_data[2][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[2][1]/synopsis_m*100) + ")",
        "4위 키워드": synopsis_data[3][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[3][1]/synopsis_m*100) + ")",
        "5위 키워드": synopsis_data[4][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[4][1]/synopsis_m*100) + ")",
        "6위 키워드": synopsis_data[5][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[5][1]/synopsis_m*100) + ")",
        "7위 키워드": synopsis_data[6][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[6][1]/synopsis_m*100) + ")",}
        #"8위 키워드": synopsis_data[7][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[7][1]/synopsis_m*100) + ")",
        #"9위 키워드": synopsis_data[8][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[8][1]/synopsis_m*100) + ")",
        #"10위 키워드": synopsis_data[9][0] + " (" + '%.2f%%'%(Counter(war_synopsis_list[i]).most_common()[9][1]/synopsis_m*100) + ")",
    war_si = war_si.append(new_data,ignore_index=True)

war_si = war_si.set_index("연도")

In [ ]:
war_si

In [ ]:
word_count_1_data=[]
word_count_2_data=[]
word_count_3_data=[]
word_count_4_data=[]
word_count_5_data=[]
word_count_6_data=[]
word_count_7_data=[]
word_count_8_data=[]
word_count_9_data=[]
word_count_10_data=[]
year_data=[]
for i in range(2022,1979,-1):
    if i in [2021,2013,2006]:
        i=i-1
    
    count_num_1 = war_synopsis_list[i].count('연합군')
    count_num_2 = war_synopsis_list[i].count('전쟁')
    count_num_3 = war_synopsis_list[i].count('세계')
    count_num_4 = war_synopsis_list[i].count('가족')
    count_num_5 = war_synopsis_list[i].count('임무')
    count_num_6 = war_synopsis_list[i].count('사람')
    count_num_7 = war_synopsis_list[i].count('공격')
    count_num_8 = war_synopsis_list[i].count('아군')
    count_num_9 = war_synopsis_list[i].count('군인')
    count_num_10 = war_synopsis_list[i].count('북한군')
    synopsis_m = sum(Counter(war_synopsis_list[i]).values())
    word_count_1_data.append(round(count_num_1/synopsis_m*100,2))
    word_count_2_data.append(round(count_num_2/synopsis_m*100,2))
    word_count_3_data.append(round(count_num_3/synopsis_m*100,2))
    word_count_4_data.append(round(count_num_4/synopsis_m*100,2))
    word_count_5_data.append(round(count_num_5/synopsis_m*100,2))
    word_count_6_data.append(round(count_num_6/synopsis_m*100,2))
    word_count_7_data.append(round(count_num_7/synopsis_m*100,2))
    word_count_8_data.append(round(count_num_8/synopsis_m*100,2))
    word_count_9_data.append(round(count_num_9/synopsis_m*100,2))
    word_count_10_data.append(round(count_num_10/synopsis_m*100,2))
    year_data.append(i)


In [ ]:

war_word_df_data={'year': year_data, '연합군': word_count_1_data, '전쟁': word_count_2_data, '세계' : word_count_3_data, '가족' : word_count_4_data, '임무' : word_count_5_data, '사람' : word_count_6_data, '공격' :word_count_7_data, '아군' : word_count_8_data, '군인' : word_count_9_data, '북한군': word_count_10_data}
war_word_df = pd.DataFrame(war_word_df_data)

war_word_df

In [ ]:
#연합군, 전쟁
plt.plot(war_word_df.year,war_word_df['연합군'],label = 'ally')
plt.plot(war_word_df.year,war_word_df['전쟁'],label = 'war')
plt.legend()
plt.show()

In [ ]:
#세계, 가족
plt.plot(war_word_df.year,war_word_df['세계'],label = 'world')
plt.plot(war_word_df.year,war_word_df['가족'],label = 'family')
plt.legend()
plt.show()

In [ ]:
#임무, 사람
plt.plot(war_word_df.year,war_word_df['임무'],label = 'mission')
plt.plot(war_word_df.year,war_word_df['사람'],label = 'people')
plt.legend()
plt.show()

In [ ]:
#공격, 아군
plt.plot(war_word_df.year,war_word_df['공격'],label = 'attack')
plt.plot(war_word_df.year,war_word_df['아군'],label = 'our soldier')
plt.legend()
plt.show()

In [ ]:
#군인, 북한군
plt.plot(war_word_df.year,war_word_df['군인'],label = 'soldier')
plt.plot(war_word_df.year,war_word_df['북한군'],label = 'North Korean soldier')
plt.legend()
plt.show()

# 뮤지컬영화

In [ ]:
#뮤지컬영화 

DF_musical=DF[DF.genre.str.contains('뮤지컬')]
DF_musical


In [ ]:
# 뮤지컬 영화 시놉시스 태깅(명사)
musical_synopsis_list = {}
for i in range(2022,1979,-1):
    musical_synopsis_list[i] = []
    sentences = DF_musical[DF.year==i]['synopsis'].values.tolist()
    tagged_sentences = [tagger.pos(sent) for sent in sentences]
    for sent in tagged_sentences:   
        for word, tag in sent:
            if tag in ['NNG'] and word not in ['시작', '한편','자신', '속']: #and len(word)>1:
                musical_synopsis_list[i].append(word)


In [ ]:
musical_synopsis_list[2021]

In [ ]:
# 리스트 틀 만들기
musical_synopsis_info = {
    "연도": [],
    "1위 키워드": [],
    "2위 키워드": [],
    "3위 키워드": [],
    "4위 키워드": [],
    "5위 키워드": [],
    "6위 키워드": [],
    "7위 키워드": [],
    "8위 키워드": [],
    "9위 키워드": [],
    "10위 키워드": []
}
musical_si = pd.DataFrame(musical_synopsis_info)

In [ ]:
musical_si

In [ ]:
for i in range(2022,1979,-1):
    if sum(Counter(musical_synopsis_list[i]).values())==0:
        print(i)

In [ ]:
for i in range(2022,1979,-1):
    if i in [2012,2010,2005,2002,1999,1989,1984,1982]:
        i=i-1
    elif i in[2013,2000,1990]:
        i=i+1
        
    synopsis_data = Counter(musical_synopsis_list[i]).most_common()
    synopsis_m = sum(Counter(musical_synopsis_list[i]).values())
    new_data = {
        "연도": str(i),
        "1위 키워드": synopsis_data[0][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[0][1]/synopsis_m*100) + ")",
        "2위 키워드": synopsis_data[1][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[1][1]/synopsis_m*100) + ")",
        "3위 키워드": synopsis_data[2][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[2][1]/synopsis_m*100) + ")",
        "4위 키워드": synopsis_data[3][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[3][1]/synopsis_m*100) + ")",
        "5위 키워드": synopsis_data[4][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[4][1]/synopsis_m*100) + ")",
        "6위 키워드": synopsis_data[5][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[5][1]/synopsis_m*100) + ")",
        "7위 키워드": synopsis_data[6][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[6][1]/synopsis_m*100) + ")",
        "8위 키워드": synopsis_data[7][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[7][1]/synopsis_m*100) + ")",
        "9위 키워드": synopsis_data[8][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[8][1]/synopsis_m*100) + ")",
        "10위 키워드": synopsis_data[9][0] + " (" + '%.2f%%'%(Counter(musical_synopsis_list[i]).most_common()[9][1]/synopsis_m*100) + ")",}
    musical_si = musical_si.append(new_data,ignore_index=True)

musical_si = musical_si.set_index("연도")

In [ ]:
musical_si

In [ ]:
for i in range(2022,1979,-1):
    if sum(Counter(musical_synopsis_list[i]).values())==0:
        print(i)

In [ ]:
word_count_1_data=[]
word_count_2_data=[]
word_count_3_data=[]
word_count_4_data=[]
word_count_5_data=[]
word_count_6_data=[]
word_count_7_data=[]
word_count_8_data=[]
word_count_9_data=[]
word_count_10_data=[]
year_data=[]

for i in range(2022,1989,-1):
    if i in [2012,2010,2005,2002,1999,1989,1984,1982]:
        count_num_1 = musical_synopsis_list[i-1].count('춤')
        count_num_2 = musical_synopsis_list[i-1].count('최고')
        count_num_3 = musical_synopsis_list[i-1].count('노래')
        count_num_4 = musical_synopsis_list[i-1].count('친구')
        count_num_5 = musical_synopsis_list[i-1].count('마을')
        count_num_6 = musical_synopsis_list[i-1].count('딸')
        count_num_7 = musical_synopsis_list[i-1].count('세상')
        count_num_8 = musical_synopsis_list[i-1].count('평화')
        count_num_9 = musical_synopsis_list[i-1].count('준비')
        count_num_10 = musical_synopsis_list[i-1].count('여자')
        synopsis_m = sum(Counter(musical_synopsis_list[i-1]).values())
        word_count_1_data.append(round(count_num_1/synopsis_m*100,2))
        word_count_2_data.append(round(count_num_2/synopsis_m*100,2))
        word_count_3_data.append(round(count_num_3/synopsis_m*100,2))
        word_count_4_data.append(round(count_num_4/synopsis_m*100,2))
        word_count_5_data.append(round(count_num_5/synopsis_m*100,2))
        word_count_6_data.append(round(count_num_6/synopsis_m*100,2))
        word_count_7_data.append(round(count_num_7/synopsis_m*100,2))
        word_count_8_data.append(round(count_num_8/synopsis_m*100,2))
        word_count_9_data.append(round(count_num_9/synopsis_m*100,2))
        word_count_10_data.append(round(count_num_10/synopsis_m*100,2))
        year_data.append(i)
        
    elif i in[2013,2000,1990]:
        count_num_1 = musical_synopsis_list[i+1].count('춤')
        count_num_2 = musical_synopsis_list[i+1].count('최고')
        count_num_3 = musical_synopsis_list[i+1].count('노래')
        count_num_4 = musical_synopsis_list[i+1].count('친구')
        count_num_5 = musical_synopsis_list[i+1].count('마을')
        count_num_6 = musical_synopsis_list[i+1].count('딸')
        count_num_7 = musical_synopsis_list[i+1].count('세상')
        count_num_8 = musical_synopsis_list[i+1].count('평화')
        count_num_9 = musical_synopsis_list[i+1].count('준비')
        count_num_10 = musical_synopsis_list[i+1].count('여자')
        synopsis_m = sum(Counter(musical_synopsis_list[i+1]).values())
        word_count_1_data.append(round(count_num_1/synopsis_m*100,2))
        word_count_2_data.append(round(count_num_2/synopsis_m*100,2))
        word_count_3_data.append(round(count_num_3/synopsis_m*100,2))
        word_count_4_data.append(round(count_num_4/synopsis_m*100,2))
        word_count_5_data.append(round(count_num_5/synopsis_m*100,2))
        word_count_6_data.append(round(count_num_6/synopsis_m*100,2))
        word_count_7_data.append(round(count_num_7/synopsis_m*100,2))
        word_count_8_data.append(round(count_num_8/synopsis_m*100,2))
        word_count_9_data.append(round(count_num_9/synopsis_m*100,2))
        word_count_10_data.append(round(count_num_10/synopsis_m*100,2))
        year_data.append(i)
        
    else:
        count_num_1 = musical_synopsis_list[i].count('춤')
        count_num_2 = musical_synopsis_list[i].count('최고')
        count_num_3 = musical_synopsis_list[i].count('노래')
        count_num_4 = musical_synopsis_list[i].count('친구')
        count_num_5 = musical_synopsis_list[i].count('마을')
        count_num_6 = musical_synopsis_list[i].count('딸')
        count_num_7 = musical_synopsis_list[i].count('세상')
        count_num_8 = musical_synopsis_list[i].count('평화')
        count_num_9 = musical_synopsis_list[i].count('준비')
        count_num_10 = musical_synopsis_list[i].count('여자')
        synopsis_m = sum(Counter(musical_synopsis_list[i]).values())
        word_count_1_data.append(round(count_num_1/synopsis_m*100,2))
        word_count_2_data.append(round(count_num_2/synopsis_m*100,2))
        word_count_3_data.append(round(count_num_3/synopsis_m*100,2))
        word_count_4_data.append(round(count_num_4/synopsis_m*100,2))
        word_count_5_data.append(round(count_num_5/synopsis_m*100,2))
        word_count_6_data.append(round(count_num_6/synopsis_m*100,2))
        word_count_7_data.append(round(count_num_7/synopsis_m*100,2))
        word_count_8_data.append(round(count_num_8/synopsis_m*100,2))
        word_count_9_data.append(round(count_num_9/synopsis_m*100,2))
        word_count_10_data.append(round(count_num_10/synopsis_m*100,2))
        year_data.append(i)


In [ ]:

musical_word_df_data={'year': year_data, '춤': word_count_1_data, '최고': word_count_2_data, '노래' : word_count_3_data, '친구' : word_count_4_data, '마을' : word_count_5_data, '딸' : word_count_6_data, '세상' :word_count_7_data, '평화' : word_count_8_data, '준비' : word_count_9_data, '여자': word_count_10_data}
musical_word_df = pd.DataFrame(musical_word_df_data)

musical_word_df

In [ ]:
#춤, 최고
plt.plot(musical_word_df.year,musical_word_df['춤'],label = 'dance')
plt.plot(musical_word_df.year,musical_word_df['최고'],label = 'great')
plt.legend()
plt.show()

In [ ]:
#노래, 친구
plt.plot(musical_word_df.year,musical_word_df['노래'],label = 'song')
plt.plot(musical_word_df.year,musical_word_df['친구'],label = 'friend')
plt.legend()
plt.show()

In [ ]:
#마을, 딸
plt.plot(musical_word_df.year,musical_word_df['마을'],label = 'twon')
plt.plot(musical_word_df.year,musical_word_df['딸'],label = 'daughter')
plt.legend()
plt.show()

In [ ]:
#세상, 평화
plt.plot(musical_word_df.year,musical_word_df['세상'],label = 'globe')
plt.plot(musical_word_df.year,musical_word_df['평화'],label = 'peace')
plt.legend()
plt.show()

In [ ]:
#준비, 여자
plt.plot(musical_word_df.year,musical_word_df['준비'],label = 'ready')
plt.plot(musical_word_df.year,musical_word_df['여자'],label = 'woman')
plt.legend()
plt.show()

# 다큐멘터리영화

In [10]:
#다큐멘터리 영화

DF_documentary=DF[DF.genre.str.contains('다큐멘터리')]
DF_documentary

title  year  audience                 date       genre  \
41                 그대가 조국  2022    333544  2022-05-25 00:00:00       다큐멘터리   
80          인생은 뷰티풀: 비타돌체  2022     52969  2022-09-07 00:00:00       다큐멘터리   
84    세븐틴 파워 오브 러브 : 더 무비  2022     48275  2022-04-20 00:00:00       다큐멘터리   
90                  나의 촛불  2022     40744  2022-02-10 00:00:00       다큐멘터리   
99            아치의 노래, 정태춘  2022     33399  2022-05-18 00:00:00       다큐멘터리   
253     브레이크 더 사일런스: 더 무비  2020    131693  2020-09-24 00:00:00       다큐멘터리   
294                  파바로티  2020     43486  2020-01-01 00:00:00       다큐멘터리   
385        브링 더 소울 : 더 무비  2019    336074  2019-08-07 00:00:00       다큐멘터리   
468                그날, 바다  2018    540558  2018-04-12 00:00:00       다큐멘터리   
538                노무현입니다  2017   1854867  2017-05-25 00:00:00       다큐멘터리   
750       님아, 그 강을 건너지 마오  2014    955149  2014-11-27 00:00:00       다큐멘터리   
816       님아, 그 강을 건너지 마오  2014   3846206  2014-11-27 00:00:00       다큐멘터리   
889                    그녀  2014    350618  2014-05-22 00:00:00       다큐멘터리   
1257                  오션스  2010    650994  2010-07-28 00:00:00  다큐멘터리, 드라마   
1296              울지마, 톤즈  2010    261822  2010-09-09 00:00:00       다큐멘터리   
1312                 워낭소리  2009   2962897  2009-01-15 00:00:00   다큐멘터리, 가족   
1586                    엠  2007    448350  2007-10-25 00:00:00       다큐멘터리   
1876               화씨 911  2004    459404  2004-07-22 00:00:00   다큐멘터리, 전쟁   
2097                 K-19  2002     79900  2002-10-03 00:00:00       다큐멘터리   
2389               쇼킹아시아2  1999     43188  1999-10-09 00:00:00       다큐멘터리   
2518                쇼킹아시아  1997    307360  1997-05-03 00:00:00       다큐멘터리   
2546            마이크로 코스모스  1996    134528  1996-12-07 00:00:00       다큐멘터리   
2690            마이크로 코스모스  1996     47640  1996-12-07 00:00:00       다큐멘터리   
2998                 말콤 X  1993     28159  1993-06-05 00:00:00       다큐멘터리   
3019                  JFK  1992    213727  1992-05-02 00:00:00       다큐멘터리   
3334              굿바이 엉클톰  1989    104632  1989-11-04 00:00:00       다큐멘터리   
3738         이소룡 일대기 생과 사  1979     50634  1979-05-05 00:00:00       다큐멘터리   
3936                  히틀러  1983     53246  1983-03-12 00:00:00       다큐멘터리   
3950       아무도 가르쳐 주지 않는다  1983     29723  1983-03-19 00:00:00       다큐멘터리   
3952            라스트 파라다이스  1983     28332  1983-09-17 00:00:00       다큐멘터리   
4139               속 몬도가네  1981     56877  1981-10-09 00:00:00       다큐멘터리   
4165              악령의 숨소리  1981     27348  1981-12-06 00:00:00       다큐멘터리   

                                                  actor         country  \
41                                                  nan            대한민국   
80             김호중(본인), 원태연(본인), 이주호(본인), 김호중, 원태연, 이주호            대한민국   
84                                                  nan            대한민국   
90             박영수(본인), 손석희(본인), 심상정(본인), 박영수, 손석희, 심상정            대한민국   
99                           정태춘(본인), 박은옥(본인), 정태춘, 박은옥            대한민국   
253                    RM(본인), 진(본인), 슈가(본인), RM, 진, 슈가            대한민국   
294                            루치아노 파바로티(본인), 루치아노 파바로티              영국   
385                    RM(본인), 진(본인), 슈가(본인), RM, 진, 슈가            대한민국   
468                                      정우성(내레이션), 정우성            대한민국   
538                                        노무현(본인), 노무현            대한민국   
750                                  조병만, 강계열, 조병만, 강계열            대한민국   
816                                  조병만, 강계열, 조병만, 강계열            대한민국   
889                                                 nan            대한민국   
1257  정보석(나레이터 - 한국어 버전), 진지희(나레이터 - 한국어 버전), 배한성(나레...             프랑스   
1296                       이금희(나레이션), 이태석(본인), 이금희, 이태석            대한민국   
1312     최원균(본인), 이삼순(본인), 최노인의 소(본인), 최원균, 이삼순, 최노인의 소            대한민국   
1586                             니콜라스 프리비데라, 니콜라스 프리비데라           아르헨티나   
1876                                 마이클 무어(본인), 마이클 무어              미국   
2097                

In [11]:
#다큐멘터리영화 시놉시스 태깅(명사)
documentary_synopsis_list = {}
for i in range(2022,1979,-1):
    documentary_synopsis_list[i] = []
    sentences = DF_documentary[DF.year==i]['synopsis'].values.tolist()
    tagged_sentences = [tagger.pos(sent) for sent in sentences]
    for sent in tagged_sentences:   
        for word, tag in sent:
            if tag in ['NNG'] and word not in ['한편', '시작']: #and len(word)>1:
                documentary_synopsis_list[i].append(word)

/var/folders/rl/pw215shj0g74sx1cnbh226f40000gn/T/ipykernel_74222/4271687198.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sentences = DF_documentary[DF.year==i]['synopsis'].values.tolist()


In [12]:
# 리스트 틀 만들기
documentary_synopsis_info = {
    "연도": [],
    "1위 키워드": [],
    "2위 키워드": [],
    "3위 키워드": [],
    "4위 키워드": [],
    "5위 키워드": [],
    "6위 키워드": [],
    "7위 키워드": [],
    "8위 키워드": [],
    "9위 키워드": [],
    "10위 키워드": []
}
documentary_si = pd.DataFrame(documentary_synopsis_info)

In [13]:
for i in range(2022,1979,-1):
    if sum(Counter(documentary_synopsis_list[i]).values())==0:
        print(i)

2021
2016
2015
2013
2012
2011
2008
2006
2005
2003
2001
2000
1998
1995
1994
1991
1990
1988
1987
1986
1985
1984
1982
1980


In [17]:
documentary_synopsis_list[2007]

['줄거리']

In [18]:
for i in range(2022,1979,-1):
    if sum(Counter(documentary_synopsis_list[i]).values())<10:
        print(i)

2021
2016
2015
2013
2012
2011
2008
2007
2006
2005
2003
2002
2001
2000
1998
1997
1995
1994
1993
1992
1991
1990
1988
1987
1986
1985
1984
1982
1981
1980


In [14]:

#for i in range(2022,1979,-1):
    #if i in [1980,1982,1988,1991,1995,2001,2006,2008,2013,2016,2021]:
        #i=i+1
    #elif i in [1984,1989,1994,1998,2000,2003,2005,2011,2015]:
        #i=i-1
    #elif i in [1987]:
        #i=i+2
    #elif i in [1985,2012]:
        #i=i-2
    #elif i in [1986]:
        #i=i-3
        
    #synopsis_data = Counter(documentary_synopsis_list[i]).most_common()
    #synopsis_m = sum(Counter(documentary_synopsis_list[i]).values())
    #new_data = {
        #"연도": str(i),
        #"1위 키워드": synopsis_data[0][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[0][1]/synopsis_m*100) + ")",
        #"2위 키워드": synopsis_data[1][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[1][1]/synopsis_m*100) + ")",
        #"3위 키워드": synopsis_data[2][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[2][1]/synopsis_m*100) + ")",
        #"4위 키워드": synopsis_data[3][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[3][1]/synopsis_m*100) + ")",
        #"5위 키워드": synopsis_data[4][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[4][1]/synopsis_m*100) + ")",
        #"6위 키워드": synopsis_data[5][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[5][1]/synopsis_m*100) + ")",
        #"7위 키워드": synopsis_data[6][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[6][1]/synopsis_m*100) + ")",
        #"8위 키워드": synopsis_data[7][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[7][1]/synopsis_m*100) + ")",
        #"9위 키워드": synopsis_data[8][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[8][1]/synopsis_m*100) + ")",
        #"10위 키워드": synopsis_data[9][0] + " (" + '%.2f%%'%(Counter(documentary_synopsis_list[i]).most_common()[9][1]/synopsis_m*100) + ")",}
    #documentary_si = documentary_si.append(new_data,ignore_index=True)

#documentary_si = documentary_si.set_index("연도")

/var/folders/rl/pw215shj0g74sx1cnbh226f40000gn/T/ipykernel_74222/3572782959.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  documentary_si = documentary_si.append(new_data,ignore_index=True)
/var/folders/rl/pw215shj0g74sx1cnbh226f40000gn/T/ipykernel_74222/3572782959.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  documentary_si = documentary_si.append(new_data,ignore_index=True)
/var/folders/rl/pw215shj0g74sx1cnbh226f40000gn/T/ipykernel_74222/3572782959.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  documentary_si = documentary_si.append(new_data,ignore_index=True)
/var/folders/rl/pw215shj0g74sx1cnbh226f40000gn/T/ipykernel_74222/3572782959.py:28: FutureWarning: The frame.append method is deprecated and will 

IndexError: list index out of range

In [15]:
#documentary_si

연도        1위 키워드        2위 키워드       3위 키워드        4위 키워드      5위 키워드  \
0   2022   이야기 (5.94%)    음악 (5.94%)   검찰 (3.96%)    영화 (2.97%)   타 (2.97%)   
1   2022   이야기 (5.94%)    음악 (5.94%)   검찰 (3.96%)    영화 (2.97%)   타 (2.97%)   
2   2020   최초 (15.00%)   투어 (10.00%)  멤버 (10.00%)  이야기 (10.00%)  세계 (5.00%)   
3   2019    팬 (14.29%)  이야기 (14.29%)   투어 (7.14%)    호흡 (7.14%)  열광 (7.14%)   
4   2018   기록 (11.11%)   그날 (11.11%)   침몰 (5.56%)    사고 (5.56%)  발생 (5.56%)   
5   2017   후보 (12.50%)    선거 (8.33%)   꼴찌 (8.33%)    국민 (8.33%)  전국 (8.33%)   
6   2017   후보 (12.50%)    선거 (8.33%)   꼴찌 (8.33%)    국민 (8.33%)  전국 (8.33%)   
7   2014  할아버지 (9.64%)   할머니 (6.02%)   서로 (4.82%)    계열 (2.41%)   로 (2.41%)   
8   2014  할아버지 (9.64%)   할머니 (6.02%)   서로 (4.82%)    계열 (2.41%)   로 (2.41%)   
9   2014  할아버지 (9.64%)   할머니 (6.02%)   서로 (4.82%)    계열 (2.41%)   로 (2.41%)   
10  2010    바다 (7.69%)     속 (5.13%)   눈물 (5.13%)    세계 (3.85%)  수단 (3.85%)   
11  2010    바다 (7.69%)     속 (5.13%)   눈물 (5.13%)    세계 (3.85%)  수단 (3.85%)   
12  2010    바다 (7.69%)     속 (5.13%)   눈물 (5.13%)    세계 (3.85%)  수단 (3.85%)   
13  2009    노인 (8.33%)    평생 (4.17%)    땅 (4.17%)     게 (4.17%)  최고 (4.17%)   
14  2009    노인 (8.33%)    평생 (4.17%)    땅 (4.17%)     게 (4.17%)  최고 (4.17%)   

         6위 키워드      7위 키워드       8위 키워드      9위 키워드     10위 키워드  
0   목소리 (2.97%)  자신 (1.98%)    틀 (1.98%)  바깥 (1.98%)  공연 (1.98%)  
1   목소리 (2.97%)  자신 (1.98%)    틀 (1.98%)  바깥 (1.98%)  공연 (1.98%)  
2    공연 (5.00%)  월간 (5.00%)    위 (5.00%)  무대 (5.00%)   뒤 (5.00%)  
3   마지막 (7.14%)  긴장 (7.14%)  다음날 (7.14%)   탑 (7.14%)   뒤 (7.14%)  
4    시간 (5.56%)  항로 (5.56%)  데이터 (5.56%)  과학 (5.56%)  분석 (5.56%)  
5    시장 (4.17%)  출마 (4.17%)   낙선 (4.17%)  대선 (4.17%)  당시 (4.17%)  
6    시장 (4.17%)  출마 (4.17%)   낙선 (4.17%)  대선 (4.17%)  당시 (4.17%)  
7    조병 (2.41%)  빛깔 (2.41%)   한복 (2.41%)   손 (2.41%)   꽃 (2.41%)  
8    조병 (2.41%)  빛깔 (2.41%)   한복 (2.41%)   손 (2.41%)   꽃 (2.41%)  
9    조병 (2.41%)  빛깔 (2.41%)   한복 (2.41%)   손 (2.41%)   꽃 (2.41%)  
10   마을 (3.85%)  신비 (2.56%)   남자 (2.56%)  사진 (2.56%)  사람 (2.56%)  
11   마을 (3.85%)  신비 (2.56%)   남자 (2.56%)  사진 (2.56%)  사람 (2.56%)  
12   마을 (3.85%)  신비 (2.56%)   남자 (2.56%)  사진 (2.56%)  사람 (2.56%)  
13  자가용 (4.17%)   귀 (4.17%)   소리 (4.17%)  귀신 (4.17%)  다리 (4.17%)  
14  자가용 (4.17%)   귀 (4.17%)   소리 (4.17%)  귀신 (4.17%)  다리 (4.17%)

In [ ]:
#word_count_1_data=[]
#word_count_2_data=[]
#word_count_3_data=[]
#word_count_4_data=[]
#word_count_5_data=[]
#word_count_6_data=[]
#word_count_7_data=[]
#word_count_8_data=[]
#word_count_9_data=[]
#word_count_10_data=[]
#year_data=[]
#for i in range(2022,1979,-1):
  #count_num_1 = documentary_synopsis_list[i].count('')
  #count_num_2 = documentary_synopsis_list[i].count('')
  #count_num_3 = documentary_synopsis_list[i].count('')
  #count_num_4 = documentary_synopsis_list[i].count('')
  #count_num_5 = documentary_synopsis_list[i].count('')
  #count_num_6 = documentary_synopsis_list[i].count('')
  #count_num_7 = documentary_synopsis_list[i].count('')
  #count_num_8 = documentary_synopsis_list[i].count('')
  #count_num_9 = documentary_synopsis_list[i].count('')
  #count_num_10 = documentary_synopsis_list[i].count('')
  #synopsis_m = sum(Counter(documentary_synopsis_list[i]).values())
  #word_count_1_data.append(round(count_num_1/synopsis_m*100,2))
  #word_count_2_data.append(round(count_num_2/synopsis_m*100,2))
  #word_count_3_data.append(round(count_num_3/synopsis_m*100,2))
  #word_count_4_data.append(round(count_num_4/synopsis_m*100,2))
  #word_count_5_data.append(round(count_num_5/synopsis_m*100,2))
  #word_count_6_data.append(round(count_num_6/synopsis_m*100,2))
  #word_count_7_data.append(round(count_num_7/synopsis_m*100,2))
  #word_count_8_data.append(round(count_num_8/synopsis_m*100,2))
  #word_count_9_data.append(round(count_num_9/synopsis_m*100,2))
  #word_count_10_data.append(round(count_num_10/synopsis_m*100,2))
  #year_data.append(i)

In [ ]:
documentary_word_df_data={'year': year_data, '': word_count_1_data, '': word_count_2_data, '' : word_count_3_data, '' : word_count_4_data, '' : word_count_5_data, '' : word_count_6_data, '' :word_count_7_data, '' : word_count_8_data, '' : word_count_9_data, '': word_count_10_data}
documentary_word_df = pd.DataFrame(documentary_word_df_data)

documentary_word_df

In [ ]:
#
plt.plot(documentary_word_df.year,documentary_word_df[''],label = '')
plt.plot(documentary_word_df.year,documentary_word_df[''],label = '')
plt.legend()
plt.show()

In [ ]:
#
plt.plot(documentary_word_df.year,documentary_word_df[''],label = '')
plt.plot(documentary_word_df.year,documentary_word_df[''],label = '')
plt.legend()
plt.show()

In [ ]:
#
plt.plot(documentary_word_df.year,documentary_word_df[''],label = '')
plt.plot(documentary_word_df.year,documentary_word_df[''],label = '')
plt.legend()
plt.show()

In [ ]:
#
plt.plot(documentary_word_df.year,documentary_word_df['신'],label = 'god')
plt.plot(documentary_word_df.year,documentary_word_df['친구'],label = 'friend')
plt.legend()
plt.show()

In [ ]:
#
plt.plot(documentary_word_df.year,documentary_word_df[''],label = '')
plt.plot(documentary_word_df.year,documentary_word_df[''],label = '')
plt.legend()
plt.show()